# Binning UVIS Full-Frame Bias Reference File

This notebook is specifically made for the WFC3/UVIS 2x2, and 3x3 binned bias reference file. However, the binning function(sci_rebin) can be used for the other various WFC3 calibration (i.e. Darks, Flats, etc.). The user will have to format the file to the appropriate shape and size, for more information about the shape and size of the binned reference files see: http://www.stsci.edu/hst/observatory/crds/SIfileInfo/WFC3/description.html

### Side Notes
- The bias binned files are made by taking the sum of the binned values

- The DQ arrays are zero

- The error arrays are propagated using the following equation
For 2x2 bin
err(binned)=sqrt(err1*err1+err2*err2+err3*err3+err4*err4)


- For more information on the binned reference files see:

http://www.stsci.edu/hst/observatory/crds/SIfileInfo/WFC3/description.html

http://www.stsci.edu/hst/wfc3/documents/ISRs/2003/WFC3-2003-14.pdf

http://www.stsci.edu/hst/wfc3/documents/TIRs/WFC3-TIR-2012-04.pdf

## Import packages

In [2]:
from astropy.io import fits
import glob
import scipy
from scipy import stats
import os
import numpy as np

In [8]:
#CD into working dir where bias data is located
#working_dir ='/Users/mmckay/Desktop/binned_bias/'
working_dir ='/Users/mmckay/Desktop/binned_bias/uvis_bias_binning/'
os.chdir(working_dir)

# Download full-frame bias reference file from CRDS website
## https://hst-crds.stsci.edu/

### Example file
25v1821hi_bia.fits - 2018 full-frame bias reference file

# Binning function (Sum/Mean)

## The main code came from:
### https://scipython.com/blog/binning-a-2d-array-in-numpy/


In [10]:
def sci_rebin(arr, new_shape, math):
    '''
    Bins an array to a specified porportional size
    
    
    Parameters:
         arr - 2D array
         
         new_shape - the desired shape of the array after binning
                    *Must be porportional to the size of the current array* 
                     (i.e 4x4 array ----> 2x2 array)
                     
         math - The calculation of the binned values (Sum or Mean)
                
                Default (sum)
         
    Example:
      
        Make an array 2x2 array
        a = np.arange(4).reshape((2, 2))
        print(a)
        
        [[0 1]
         [2 3]]
        
        Bin the 2x2 array
        
        b = sci_rebin(a, [1,1], math='sum')
        print(b)
        
        [[6]]
        
    
    '''
    shape1 = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1])
    if math == 'sum':
        bin_values = arr.reshape(shape1).sum(-1).sum(1)
        return bin_values
    elif math == 'mean':
        bin_values = arr.reshape(shape1).mean(-1).mean(1)
        return bin_values
    else:
        bin_values = arr.reshape(shape1).sum(-1).sum(1)
        return bin_values
    
    


# Processing full-frame image function

In [11]:
def uvis_ref_binning(unbinned_ref_files, binning, math):
    '''
    Bins WFC3/UVIS full-frame SCI, DQ and Err arrays
    
    The Err arrays propagated depending on the the calculation 
    of the binned values(sum or mean)
    
    (sum) binned error pixel= sqrt(err1*err1+err2*err2+err3*err3+err4*err4...+err9*err9)
    (mean) binned error pixel= 1/9 sqrt(err1*err1+err2*err2+err3*err3+err4*err4...+err9*err9)
    
    Parameters:
         unbinned_ref_files - Standard WFC3/UVIS full frame bias file with 
         serial and vertial overscan
         
         binning - The binning mode (2 for 2x2 or 3 for 3x3)
         
         math - The calculation of the binned values (Sum or Mean)
    
    Output:
    
        New FITS file with binned data (6 extensions)
        Filename: final_2x2bin_2009__bia.fits
        No.    Name      Ver    Type      Cards   Dimensions   Format
          0  PRIMARY       1 PrimaryHDU     730   ()      
          1  SCI           1 ImageHDU        21   (2102, 1035)   float32   
          2  ERR           1 ImageHDU        20   (2102, 1035)   float32   
          3  DQ            1 ImageHDU        20   (2102, 1035)   int16   
          4  SCI           2 ImageHDU        21   (2102, 1035)   float32   
          5  ERR           2 ImageHDU        20   (2102, 1035)   float32   
          6  DQ            2 ImageHDU        20   (2102, 1035)   int16  
        
        
    Example:
        2x2 binning (mean) a full-frame bias image 
        uvis_ref_binning(2010a_final_bias_ref.fits, 2, math = 'mean')
        
    
    '''
    if binning == 3:
        nrow=690
        ncol=1402
        N=9
    
    elif binning == 2:
        nrow=1035
        ncol=2102
        N=4
        
    for im in unbinned_ref_files:
        unbinned_im=fits.open(im,mode='update')
        ref_ext1=unbinned_im[1].data
        ref_ext2=unbinned_im[2].data
        ref_ext3=unbinned_im[3].data
        ref_ext4=unbinned_im[4].data
        ref_ext5=unbinned_im[5].data
        ref_ext6=unbinned_im[6].data
        
        print(ref_ext1.shape)
        
        #Binning the fullframe bias files  
        #Sci extensions
        binned_ext1=sci_rebin(ref_ext1,[nrow,ncol],math)
        binned_ext4=sci_rebin(ref_ext4,[nrow,ncol],math)
        
        print(binned_ext1.shape)
        
        #DQ extensions
        binned_ext3=sci_rebin(ref_ext3,[nrow,ncol],math)
        binned_ext6=sci_rebin(ref_ext6,[nrow,ncol],math)
    
        
        #Error extensions
        #Calculate new errors for binned 
        #Square the errors
        unbinned_err_box=ref_ext2[600:603,600:603]
        print('unbinned error box[600:603, 600:603], extension = 2')
        print(unbinned_err_box)
        print('')
        
        sq_ext2=ref_ext2**2
        sq_ext5=ref_ext5**2
        
        sq_unbinned=sq_ext2[600:603,600:603]
        print('squared unbinned error values [600:603, 600:603], extension = 2')
        print(sq_unbinned)
        print('')
        
        #Sum of the square error
        sqbin_ext2=sci_rebin(sq_ext2,[nrow,ncol],math='sum')
        sqbin_ext5=sci_rebin(sq_ext5,[nrow,ncol],math='sum')
        
        sq_binned=sqbin_ext2[200:201,200:201]
        print('binned(sum) squared values [200:201,200:201], extension = 2')
        print(sq_binned)
        print('')
        
        #Square root of the sum and divide by the number of unbinned pix
        
        if math == 'mean':
            print('sqrt value is divided by number of binned pixels(i.e 2x2 binned N = 4)')
            sqrt_sqbin_ext2 = np.sqrt(sqbin_ext2) / N
            sqrt_sqbin_ext5 = np.sqrt(sqbin_ext5) / N
            
        else:
            sqrt_sqbin_ext2 = np.sqrt(sqbin_ext2)
            sqrt_sqbin_ext5 = np.sqrt(sqbin_ext5) 
        
        sqrt_sqbin_unbinned = sqrt_sqbin_ext2[200:201,200:201]
        print('sqrt of binned squared values, extension = 2')
        print(sqrt_sqbin_unbinned)
        print('')
        
        #Change variable
        binned_ext2 = sqrt_sqbin_ext2
        binned_ext5 = sqrt_sqbin_ext5
        
        ##--------------------------------------------------------------------------------
         
        unbinned_im.close()
    
        #Create new fits file with binned data
        new_hdul = fits.HDUList()
        new_hdul.append(fits.PrimaryHDU())
        new_hdul.append(fits.ImageHDU(binned_ext1))
        new_hdul.append(fits.ImageHDU(binned_ext2))
        new_hdul.append(fits.ImageHDU(binned_ext3))
        new_hdul.append(fits.ImageHDU(binned_ext4))
        new_hdul.append(fits.ImageHDU(binned_ext5))
        new_hdul.append(fits.ImageHDU(binned_ext6))
        new_hdul.writeto('{}x{}bin.fits'.format(binning,binning),overwrite=True)
    

## 2x2 binning 
### UVIS unbinned full frame with overscan size (2070, 4206) 
### UVIS 2x2 binned images with overscan size (1035, 2102)

- 2 columns are removed from the full frame serial overscan to adjust image to fit the correct size for the 2x2 array 1035,2102.

- The mixed columns are not removed

- Note about the bias reference files
- The full-frame bias reference files has zero rows near the overscan. Chip1 science row 2049:2052 (3 rows) and Chip2 science row 20:23 (3 rows)

# Remove columns from serial overscan

### Important note (ONLY RUN ONCE!!!)
When you run this part of the script it permanently removes two virtual overscan columns. If executed multiple times on the same file, two more columns will be deleted. Make sure to have a directory with clean files in case you make a mistake.

In [12]:
unbin_ref=glob.glob('*.fits')

for i in unbin_ref:
    hdu = fits.open(i,mode='update')
    chip2_sci = hdu[1].data
    chip2_err = hdu[2].data
    chip2_dq =  hdu[3].data
    chip1_sci = hdu[4].data
    chip1_err = hdu[5].data
    chip1_dq =  hdu[6].data
    
    #Remove 2 cols from the virtual overscan
    
    hdu[1].data = np.delete(hdu[1].data, 2100,1)
    hdu[2].data = np.delete(hdu[2].data, 2100,1)
    hdu[3].data = np.delete(hdu[3].data, 2100,1)
    hdu[4].data = np.delete(hdu[4].data, 2100,1)
    hdu[5].data = np.delete(hdu[5].data, 2100,1)
    hdu[6].data = np.delete(hdu[6].data, 2100,1)
    
    hdu[1].data = np.delete(hdu[1].data, 2100,1)
    hdu[2].data = np.delete(hdu[2].data, 2100,1)
    hdu[3].data = np.delete(hdu[3].data, 2100,1)
    hdu[4].data = np.delete(hdu[4].data, 2100,1)
    hdu[5].data = np.delete(hdu[5].data, 2100,1)
    hdu[6].data = np.delete(hdu[6].data, 2100,1)
    
    print(hdu[1].shape)    
    hdu.close()



(2070, 4204)


# Run binning code

In [13]:
unbin_ref =  glob.glob('*.fits')
uvis_ref_binning(unbin_ref, 2, math = 'sum')

(2070, 4204)
(1035, 2102)
unbinned error box[600:603, 600:603], extension = 2
[[0.13372174 0.13412371 0.13358879]
 [0.13391082 0.13200374 0.13456431]
 [0.13428088 0.1336813  0.13199793]]

squared unbinned error values [600:603, 600:603], extension = 2
[[0.0178815  0.01798917 0.01784597]
 [0.01793211 0.01742499 0.01810755]
 [0.01803135 0.01787069 0.01742345]]

binned(sum) squared values [200:201,200:201], extension = 2
[[0.07152131]]

sqrt value is divided by number of binned pixels(i.e 2x2 binned N = 4)
sqrt of binned squared values, extension = 2
[[0.06685867]]



# Replace mixed columns with zeros
- Due to the odd number of columns in the physical overscan, exposed columns next to the overscan are binned resulting a lower value.
- We set the values of the mixed columns to zeros

In [14]:
bin_2x2_files = glob.glob('2x2*.fits')

for im in bin_2x2_files:
    year= im[4:9]
    hdu = fits.open(im,mode='update')
    
    sci_data1 = hdu[1].data
    err_data1 = hdu[2].data
    dqa_data1 = hdu[3].data
    
    sci_data2 = hdu[4].data
    err_data2 = hdu[5].data
    dqa_data2 = hdu[6].data
    
    #Set mixed columns to zero
    sci_data1[:,12] = 0
    sci_data1[:,2089] = 0
    err_data1[:,12] = 0
    err_data1[:,2089] = 0
    dqa_data1[:,12] = 0
    dqa_data1[:,2089] = 0
    
    sci_data2[:,12] = 0
    sci_data2[:,2089] = 0
    err_data2[:,12] = 0
    err_data2[:,2089] = 0
    dqa_data2[:,12] = 0
    dqa_data2[:,2089] = 0  
    
    
    hdu[1].data = sci_data1 
    hdu[2].data = err_data1 
    hdu[3].data = dqa_data1 
    hdu[4].data = sci_data2 
    hdu[5].data = err_data2 
    hdu[6].data = dqa_data2 
    
    
    hdu.close()

## 3x3 binning 
### UVIS unbinned full frame with overscan size (2070, 4206) 
### UVIS 3x3 binned images with overscan size (690 1402)

- No columns are removed to perform the 3x3 binning
- the mixed colums are not removed

In [25]:
working_dir ='/Users/mmckay/Desktop/binned_bias/test_code/'
os.chdir(working_dir)

# Run binning code

In [26]:
unbin_ref=glob.glob('2010a_final_bias_ref 2.fits')
uvis_ref_binning(unbin_ref,3, math='sum')

(2070, 4206)
(690, 1402)
unbinned error box[600:603, 600:603], extension = 2
[[0.09497067 0.09461706 0.09532122]
 [0.09441956 0.09462713 0.09560762]
 [0.09481271 0.09490033 0.09551942]]

squared unbinned error values [600:603, 600:603], extension = 2
[[0.00901943 0.00895239 0.00908614]
 [0.00891505 0.00895429 0.00914082]
 [0.00898945 0.00900607 0.00912396]]

binned(sum) squared values [200:201,200:201], extension = 2
[[0.0811876]]

sqrt of binned squared values, extension = 2
[[0.28493437]]



# Replace mixed columns with zeros
- Due to the odd number of columns in the physical overscan, exposed columns next to the overscan are binned resulting a lower value.
- We set the values of the mixed columns to zeros

In [27]:
bin_3x3_files = glob.glob('3x3*.fits')

for im in bin_3x3_files:
    hdu = fits.open(im,mode='update')
    
    sci_data1 = hdu[1].data
    err_data1 = hdu[2].data
    dqa_data1 = hdu[3].data
    
    sci_data2 = hdu[4].data
    err_data2 = hdu[5].data
    dqa_data2 = hdu[6].data

    print(sci_data1[300:302,7:9])
    print(sci_data1[300:302,1393:1395])
    
    
    
    #Set mixed columns to zero
    sci_data1[:,8] = 0
    sci_data1[:,1393] = 0
    err_data1[:,8] = 0
    err_data1[:,1393] = 0
    dqa_data1[:,8] = 0
    dqa_data1[:,1393] = 0
    
    sci_data2[:,8] = 0
    sci_data2[:,1393] = 0
    err_data2[:,8] = 0
    err_data2[:,1393] = 0
    dqa_data2[:,8] = 0
    dqa_data2[:,1393] = 0
    
    
    hdu[1].data = sci_data1 
    hdu[2].data = err_data1 
    hdu[3].data = dqa_data1 
    hdu[4].data = sci_data2 
    hdu[5].data = err_data2 
    hdu[6].data = dqa_data2 
    
    print(sci_data1[300:302,7:9])
    print(sci_data1[300:302,1393:1395])
    hdu.close()   

[[0.        1.2090609]
 [0.        0.7061602]]
[[-3.6933045  0.       ]
 [-3.589984   0.       ]]
[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]
